Actor-critic algortythm for 2048 game

In [13]:
import tensorflow as tf
import numpy as np

In [20]:
game_board = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
score = 0

In [42]:
import logic_2048
import time
import random

# Hyperparameters
learning_rate = 0.001
discount_factor = 0.99
epsilon = 0.1

# Initialize the game
game_board = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
game_matrix = logic_2048.place_new(game_board)
score = 0
n = 0

# Function to evaluate the state and action
def evaluate_state_action(state, action):

    rows = len(state)
    cols = len(state[0])

    score = 0

    sum_values = sum(sum(row) for row in state)
    score += sum_values

    max_value = max(max(row) for row in state)
    score += max_value

    num_empty_cells = sum(row.count(0) for row in state)
    score += num_empty_cells

    if action == "a":
        # 1. Favor states where the larger values are concentrated on the left side
        left_sum = sum(state[row][col] for row in range(rows) for col in range(cols // 2))
        right_sum = sum(state[row][col] for row in range(rows) for col in range(cols // 2, cols))
        score += (left_sum - right_sum)

        # 2. Give higher scores for states where adjacent cells have similar values
        adjacency_bonus = 0
        for row in range(rows):
            for col in range(cols - 1):
                if state[row][col] == state[row][col + 1]:
                    adjacency_bonus += state[row][col]
        score += adjacency_bonus

    elif action == "s":
        # 1. Favor states where the larger values are concentrated at the bottom
        bottom_sum = sum(state[row][col] for row in range(rows // 2, rows) for col in range(cols))
        top_sum = sum(state[row][col] for row in range(rows // 2) for col in range(cols))
        score += (bottom_sum - top_sum)

        # 2. Give higher scores for states where adjacent cells have similar values
        adjacency_bonus = 0
        for row in range(rows - 1):
            for col in range(cols):
                if state[row][col] == state[row + 1][col]:
                    adjacency_bonus += state[row][col]
        score += adjacency_bonus

    elif action == "w":
        # 1. Favor states where the larger values are concentrated at the top
        top_sum = sum(state[row][col] for row in range(rows // 2) for col in range(cols))
        bottom_sum = sum(state[row][col] for row in range(rows // 2, rows) for col in range(cols))
        score += (top_sum - bottom_sum)

        # 2. Give higher scores for states where adjacent cells have similar values
        adjacency_bonus = 0
        for row in range(1, rows):
            for col in range(cols):
                if state[row][col] == state[row - 1][col]:
                    adjacency_bonus += state[row][col]
        score += adjacency_bonus

    elif action == "d":
        # 1. Favor states where the larger values are concentrated on the right side
        right_sum = sum(state[row][col] for row in range(rows) for col in range(cols // 2, cols))
        left_sum = sum(state[row][col] for row in range(rows) for col in range(cols // 2))
        score += (right_sum - left_sum)

        # 2. Give higher scores for states where adjacent cells have similar values
        adjacency_bonus = 0
        for row in range(rows):
            for col in range(cols - 1):
                if state[row][col] == state[row][col + 1]:
                    adjacency_bonus += state[row][col]
        score += adjacency_bonus

    return score

def choose_action(state, epsilon):
    if random.random() < epsilon:
        # Explore: Random action is chosen
        action = random.choice(["a", "s", "w", "d"])
    else:
        # Exploit: Choose the action with the highest evaluation
        action_values = {
            "a": evaluate_state_action(state, "a"),
            "s": evaluate_state_action(state, "s"),
            "w": evaluate_state_action(state, "w"),
            "d": evaluate_state_action(state, "d")
        }
        action = max(action_values, key=action_values.get)
    return action

# Training loop
while True:
    state = game_matrix

    # Choose action based on the current state and policy (Actor model)
    action = choose_action(state, epsilon)

    # Apply the chosen action to the game and observe the new state, reward, and whether the game is over
    new_game_matrix, reward, success = logic_2048.transform_matrix(game_matrix, action)

    # Update the game matrix, score, and check if the game is over or won
    game_matrix = new_game_matrix
    score += reward

    if logic_2048.win_check(game_matrix):
        print("CONGRATS! YOU WON!")
        print(str(game_matrix) + "\nScore: " + str(score))
        break
    elif not success:
        print("Game over")
        print(str(game_matrix) + "\nScore: " + str(score))
        break

    # Compute the target value for the Critic model
    next_state = game_matrix
    next_action = None  # Choose the best action based on the evaluation
    next_state_value = evaluate_state_action(next_state, next_action)
    target_value = reward + discount_factor * next_state_value

    # Compute the predicted value for the current state (Critic model)
    predicted_value = evaluate_state_action(state, action)

    # Compute the TD error
    td_error = target_value - predicted_value

    #ToDo:
        # Update the Critic model
        # Perform gradient descent to minimize the TD error between the predicted value and target value

        # Update the Actor model
        # Perform gradient ascent to maximize the value of the chosen action in the current state

    if logic_2048.win_check(game_matrix):
        print("CONGRATS! YOU WON!")
        print(str(game_matrix) + "\nScore: " + str(score))
        break

    elif not success:
        print("Game over")
        print(str(game_matrix) + "\nScore: " + str(score))
        break


    n += 1
    print(game_matrix)
    time.sleep(0.05)  # Introduce a delay before


[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [2, 0, 0, 2]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [2, 0, 0, 2]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [2, 0, 0, 2]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [2, 0, 0, 2]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, 4]]
[[0, 0, 0, 0], [0, 0, 0, 2], [0, 0, 0, 0], [2, 4, 0, 0]]
[[0, 0, 2, 0], [2, 0, 0, 0], [0, 0, 0, 0], [2, 4, 0, 0]]
[[2, 0, 0, 0], [2, 0, 0, 0], [0

KeyboardInterrupt: ignored